<a href="https://colab.research.google.com/github/gizelleguerra/genAI_demos/blob/main/Tivdak_chatbot_gpt_3_5_turbo_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!mkdir -p DATA/DORA;curl https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:32022R2554 -o DATA/DORA/CELEX_32022R2554.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1458k    0 1458k    0     0  1536k      0 --:--:-- --:--:-- --:--:-- 1536k


In [1]:
!mkdir -p DATA/TIVDAK;curl https://docs.seagen.com/Tivdak_Full_Ltr_Master.pdf -o DATA/TIVDAK/tivdak.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  452k  100  452k    0     0   465k      0 --:--:-- --:--:-- --:--:--  465k


In [2]:
%pip install -q llama-index openai pypdf gradio loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.2/626.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.8/269.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.2/294.2 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.5 MB/s eta 0:00

In [3]:
import gradio as gr
import loguru

import llama_index
from llama_index import (
    ListIndex, VectorStoreIndex, SimpleDirectoryReader, LLMPredictor,
    ServiceContext,StorageContext, load_index_from_storage
)
from llama_index.response.notebook_utils import display_response
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import logging, sys, os, json
from pathlib import Path
import openai
from typing import List, Tuple, Dict, Union

from loguru import logger


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
cwd

PosixPath('/content')

In [ ]:
# cwd: Path = Path.cwd()
# data_dir: Path = cwd / "DATA"; data_dir.mkdir(exist_ok=True, parents=True)
# dora_dir: Path  = data_dir / "DORA"; dora_dir.mkdir(exist_ok=True, parents=True)
# test_dir: Path = cwd / "TEST"; test_dir.mkdir(exist_ok=True, parents=True)
# # Load TEST
# documents: List = SimpleDirectoryReader(test_dir).load_data()

In [9]:
cwd: Path = Path.cwd()
data_dir: Path = cwd / "DATA"; data_dir.mkdir(exist_ok=True, parents=True)
drug_dir: Path  = data_dir / "TIVDAK"; drug_dir.mkdir(exist_ok=True, parents=True)
# Load data
documents: List = SimpleDirectoryReader(drug_dir).load_data()

In [10]:
def set_open_ai_key(env_path: Union[Path, str] = None) -> Tuple[bool, str]:
  import json, os
  from pathlib import Path

  openai.api_key = None #clear previous key if exists
  env_path: Path = Path(env_path).absolute() # in case env_path is passed as a str
  if not env_path.is_file():
    err: str = f"File:{env_path} does not exist."
  else:
    try:
      with open(env_path, "r") as f:
          env_vars = json.load(f)
      os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
      openai.api_key = os.environ["OPENAI_API_KEY"]
      openai.Model.list() #test a random command on the openai API
      err = None
    except Exception as err:
      logger.error(json.dumps(err))

  return (True, "ok") if not err else (False, err)

In [11]:
#cwd: Path = Path.cwd()
env_path: Path = "/content/drive/MyDrive/Colab Notebooks/openai.env";
print(set_open_ai_key(env_path)) # Tuple[bool, str]

(True, 'ok')


In [12]:
TEMPERATURE = 0

llm: llama_index.llms.openai.OpenAI = OpenAI(
    temperature=TEMPERATURE,
    model="gpt-3.5-turbo",
    max_tokens=300,
    max_retries = 2)

service_context: llama_index.indices.service_context.ServiceContext = (
    ServiceContext.from_defaults(llm=llm)
    )
index: llama_index.indices.vector_store.base.VectorStoreIndex = (
     VectorStoreIndex.from_documents(documents, service_context=service_context)
)

In [13]:
TEST = False

persist_dir = (test_dir / "storage") if TEST else (cwd / 'storage')
index.storage_context.persist(persist_dir=persist_dir)

In [15]:
# store and reload index from memory
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)

In [16]:
engine = index.as_chat_engine()#(chat_mode="openai", verbose=True)
q: str = 'Look only at the document. What is Tivdak used for?'
response: llama_index.response.schema.Response = engine.query(q)
display_response(response)

**`Final Response:`** Tivdak is used to treat adults with cervical cancer that has returned or spread to other parts of the body, and who have received chemotherapy that did not work or is no longer working.

In [ ]:
q = "You are an expert HR manager and the document is a resume. Does the candidate have experience with Sarbanes Oxley?"
display_response(engine.query(q))

**`Final Response:`** Yes, the candidate does have experience with Sarbanes Oxley. They insured compliance for Sarbanes Oxley audits by establishing and maintaining an out-of-cycle capital approval process and authorized over $50M in requests yearly.

In [ ]:
engine.query(q)

Response(response='Yes, the candidate does have experience with Sarbanes Oxley. They insured compliance for Sarbanes Oxley audits by establishing and maintaining an out-of-cycle capital approval process and authorized over $50M in requests yearly.', source_nodes=[], metadata=None)

In [17]:
from llama_index.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1000)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt="""You are a chatbot, able to have normal interactions.
    You are an assistant to medical practitioners who is able to answer questions on drug prescribing guidelines.
    """,
)

In [19]:
#response = chat_engine.chat("Does the candidate have experience with Sarbanes Oxley?");print(response)

In [18]:
response = chat_engine.chat("What kind of illness is Tivdak prescribed for?");print(response)

 TIVDAK is a prescription medicine used to treat adults with cervical cancer that has returned or has spread to other parts of the body, and who have received chemotherapy that did not work or is no longer working.


In [20]:
type(response.response)

str

In [21]:
def respond_llm(message, chat_history):
        bot_message = chat_engine.chat(message)
        chat_history.append((message, bot_message.response))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=130) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond_llm, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Allows you to Press the <submit> button
    msg.submit(respond_llm, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Allows you to Press enter to submit
gr.close_all()
demo.launch(debug=True)

# to start a new chat, add a button that calls chat_engine.reset(). It should also call clear!

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
